In [2]:
import nltk
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
nltk.download('popular')
# Compare to tokens with no cleaning
# df_tokens = pd.DataFrame.from_dict(nltk.FreqDist(tokens), orient='index')
# df_tokens.columns = ['Frequency']
# df_tokens.index.name = 'Term'

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/fabien/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /Users/fabien/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /Users/fabien/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /Users/fabien/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /Users/fabien/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /Users/fabien/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading packag

True

In [31]:
link = "./rotten_tomatoes_critic_reviews.csv"
link2 ='./rotten_tomatoes_movies.csv'
# method to convert format rate 3/5 to float
def get_score(rate):
    if type(rate) == str:
        r = rate.split("/")[0]
        if len(r) > 0:
            return float(r)
        
critic_reviews = pd.read_csv(link, converters={'review_date': pd.to_datetime})
movies = pd.read_csv(link2, converters={'original_release_date': pd.to_datetime, 'streaming_release_date': pd.to_datetime})

In [117]:
critic_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1130017 entries, 0 to 1130016
Data columns (total 9 columns):
 #   Column                Non-Null Count    Dtype         
---  ------                --------------    -----         
 0   rotten_tomatoes_link  1130017 non-null  object        
 1   critic_name           1111488 non-null  object        
 2   top_critic            1130017 non-null  bool          
 3   publisher_name        1130017 non-null  object        
 4   review_type           1130017 non-null  object        
 5   review_score          824081 non-null   object        
 6   review_date           1130017 non-null  datetime64[ns]
 7   review_content        1064211 non-null  object        
 8   review_score_float    822481 non-null   float64       
dtypes: bool(1), datetime64[ns](1), float64(1), object(6)
memory usage: 70.0+ MB


In [28]:
critic_reviews.head()

,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content
0,m/0814255,Andrew L. Urban,False,Urban Cinefile,Fresh,NaN,2010-02-06,A fantasy adventure that fuses Greek mythology...
1,m/0814255,Louise Keller,False,Urban Cinefile,Fresh,NaN,2010-02-06,"Uma Thurman as Medusa, the gorgon with a coiff..."
2,m/0814255,NaN,False,FILMINK (Australia),Fresh,NaN,2010-02-09,With a top-notch cast and dazzling special eff...
3,m/0814255,Ben McEachen,False,Sunday Mail (Australia),Fresh,3.5/5,2010-02-09,Whether audiences will get behind The Lightnin...
4,m/0814255,Ethan Alter,True,Hollywood Reporter,Rotten,NaN,2010-02-10,What's really lacking in The Lightning Thief i...


In [138]:
def get_score(x):
    x = x.replace(" ", "")
    letters = {"A":14, "B":11, "C":8,"D":5,"E":2, "F":0}
        #convert string "3.4/5" to rate on 100
    if (type(x)==str) and ('/' in x) and (float(x.split("/")[1]) > float(x.split("/")[0])):
        return 100*(float(x.split("/")[0]) / float(x.split("/")[1]))
    elif x[0] in letters.keys():
        #convert letters rate to rate on 100
        return 100*((letters[x[0]] + 44 - ord("-") if len(x)>1 and x[1] in "-+" else letters[x[0]])/15)

critic_reviews["review_score_float"] = critic_reviews[critic_reviews["review_score"].isna()==False]["review_score"].apply(get_score)
critic_reviews[critic_reviews["review_score_float"].isna()==False]["review_score_float"]

3          70.000000
6          25.000000
7          70.000000
8          73.333333
9          60.000000
             ...    
1130012    40.000000
1130013    70.000000
1130014    66.666667
1130015    70.000000
1130016    53.333333
Name: review_score_float, Length: 777569, dtype: float64

In [139]:
critic_reviews[(critic_reviews["review_score_float"].isna() != True)][["review_score", "review_score_float"]]

,review_score,review_score_float
3,3.5/5,70.000000
6,1/4,25.000000
7,3.5/5,70.000000
8,B,73.333333
9,3/5,60.000000
...,...,...
1130012,2/5,40.000000
1130013,3.5/5,70.000000
1130014,B+,66.666667
1130015,3.5/5,70.000000


In [140]:
critic_reviews.describe()

,review_score_float
count,777569.000000
mean,60.950732
std,19.417832
min,0.000000
25%,50.000000
50%,62.500000
75%,75.000000
max,99.900000


In [137]:
# Check why value max is 900
critic_reviews[critic_reviews["review_score_float"] > 100]

,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content,review_score_float
44843,m/1098652-in_crowd,Scott A. Mantz,False,Scott Mantz' Movie Reviews,Rotten,7/5,2000-01-01,If you're looking for an engaging movie with c...,140.000000
111377,m/adoration,Pam Grady,False,Boxoffice Magazine,Fresh,4.5/4,2009-04-23,Egoyan's finest in years.,112.500000
112814,m/affliction,Russell Smith,False,Austin Chronicle,Fresh,5.5/5,2000-01-01,NaN,110.000000
126588,m/american_beauty,Scott A. Mantz,False,Scott Mantz' Movie Reviews,Fresh,9/5,2000-01-01,You can't help but wonder how people find them...,180.000000
152047,m/avengers_age_of_ultron,Kevin A. Ranson,False,MovieCrypt.com,Fresh,3.5/3,2015-05-03,"... tops the original ... yet, somehow it felt...",116.666667
190365,m/bless_the_child,Scott A. Mantz,False,Scott Mantz' Movie Reviews,Rotten,8/5,2000-01-01,Considering how jaded moviegoers are these day...,160.000000
201589,m/boy_erased,Tiffany Tchobanian,False,Film Threat,Fresh,9.5/5,2018-10-16,Boy Erased is a poignant family drama that exp...,190.000000
202512,m/boys_dont_cry,Marjorie Baumgarten,False,Austin Chronicle,Fresh,5.5/5,2000-01-01,NaN,110.000000
238191,m/children_of_men,Lawrence Toppman,False,Charlotte Observer,Fresh,35/4,2007-01-04,It's as if Cuaron sees a future so hellish tha...,875.000000
238833,m/chill_factor,Scott A. Mantz,False,Scott Mantz' Movie Reviews,Rotten,9/5,2000-01-01,"Cuba, got a minute? We've gotta talk! Your car...",180.000000


In [143]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17712 entries, 0 to 17711
Data columns (total 22 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   rotten_tomatoes_link              17712 non-null  object        
 1   movie_title                       17712 non-null  object        
 2   movie_info                        17391 non-null  object        
 3   critics_consensus                 9134 non-null   object        
 4   content_rating                    17712 non-null  object        
 5   genres                            17693 non-null  object        
 6   directors                         17518 non-null  object        
 7   authors                           16170 non-null  object        
 8   actors                            17360 non-null  object        
 9   original_release_date             16546 non-null  datetime64[ns]
 10  streaming_release_date            17328 non-nu

In [145]:
movies.describe()

,runtime,tomatometer_rating,tomatometer_count,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count
count,17398.000000,17668.000000,17668.000000,17416.000000,1.741500e+04,17712.000000,17712.000000,17712.000000
mean,102.214048,60.884763,57.139801,60.554260,1.439401e+05,14.586326,36.374831,20.703139
std,18.702511,28.443348,68.370047,20.543369,1.763577e+06,15.146349,52.601038,30.248435
min,5.000000,0.000000,5.000000,0.000000,5.000000e+00,0.000000,0.000000,0.000000
25%,90.000000,38.000000,12.000000,45.000000,7.075000e+02,3.000000,6.000000,3.000000
50%,99.000000,67.000000,28.000000,63.000000,4.277000e+03,8.000000,16.000000,8.000000
75%,111.000000,86.000000,75.000000,78.000000,2.498800e+04,23.000000,44.000000,24.000000
max,266.000000,100.000000,574.000000,100.000000,3.579764e+07,69.000000,497.000000,303.000000


In [141]:
movies.head(3)

,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,content_rating,genres,directors,authors,actors,original_release_date,...,production_company,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",Though it may seem like just another Harry Pot...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,...,20th Century Fox,Rotten,49.0,149.0,Spilled,53.0,254421.0,43,73,76
1,m/0878835,Please Give,Kate (Catherine Keener) and her husband Alex (...,Nicole Holofcener's newest might seem slight i...,R,Comedy,Nicole Holofcener,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R...",2010-04-30,...,Sony Pictures Classics,Certified-Fresh,87.0,142.0,Upright,64.0,11574.0,44,123,19
2,m/10,10,"A successful, middle-aged Hollywood songwriter...",Blake Edwards' bawdy comedy may not score a pe...,R,"Comedy, Romance",Blake Edwards,Blake Edwards,"Dudley Moore, Bo Derek, Julie Andrews, Robert ...",1979-10-05,...,Waner Bros.,Fresh,67.0,24.0,Spilled,53.0,14684.0,2,16,8


In [175]:
pd.pivot_table(movies, index = ["audience_status", 'tomatometer_status'], values=["tomatometer_rating","tomatometer_count","audience_rating","audience_count", "diff_rate_audience_tomatometter"])

audience_count  audience_rating  \
audience_status tomatometer_status                                    
Spilled         Certified-Fresh      497924.536023        50.435159   
                Fresh                 49059.782410        46.846975   
                Rotten               111214.941325        38.883369   
Upright         Certified-Fresh      371180.449051        79.886859   
                Fresh                 60322.781666        76.433233   
                Rotten               149396.394149        70.638490   

                                    diff_rate_audience_tomatometter  \
audience_status tomatometer_status                                    
Spilled         Certified-Fresh                          -31.379412   
                Fresh                                    -21.308411   
                Rotten                                    10.081733   
Upright         Certified-Fresh                           -8.056252   
                Fresh                                     -2.283197   
                Rotten                                    29.671518   

                                    tomatometer_count  tomatometer_rating  
audience_status tomatometer_status                                         
Spilled         Certified-Fresh            112.000000           81.766571  
                Fresh                       34.993391           73.514489  
                Rotten                      50.378870           30.025738  
Upright         Certified-Fresh            119.510190           87.966268  
                Fresh                       31.530056           82.334908  
                Rotten                      54.920787           39.137161

In [174]:
pd.pivot_table(movies, index = ['audience_status'], values=["tomatometer_rating","tomatometer_count","audience_rating","audience_count"])

,audience_count,audience_rating,tomatometer_count,tomatometer_rating
audience_status,,,,
Spilled,112673.935357,41.375921,49.250445,43.176493
Upright,172353.678880,76.320980,62.898135,75.384656


In [178]:
#movies.drop("diff_rate_audience_tomatometter", axis=1, inplace=True)
movies["diff_rate_audience_tomatometter"] = movies[movies['critics_consensus'].isna() != True]["audience_rating"] - movies[movies['critics_consensus'].isna() != True]["tomatometer_rating"]
movies[movies['diff_rate_audience_tomatometter'].isna() != True].sort_values(by=['diff_rate_audience_tomatometter'], ascending=False)[["movie_info","critics_consensus", "audience_rating","tomatometer_rating", "diff_rate_audience_tomatometter"]]
#movies.info()

,movie_info,critics_consensus,audience_rating,tomatometer_rating,diff_rate_audience_tomatometter
7760,Filmmaker Dinesh D'Souza examines presidential...,Hillary's America: The Secret History of the D...,80.0,4.0,76.0
11487,"Combining outrageous, sexy comedy with the hot...","A party movie that substitutes surfs for snow,...",84.0,8.0,76.0
13916,"Kicked out by his parents, a gay teenager (Jer...","As an ordinary coming-of-age drama, Stonewall ...",85.0,9.0,76.0
2619,"Political commentator, author and filmmaker Di...","Passionate but poorly constructed, America pre...",84.0,8.0,76.0
10504,"In early 20th-century Paris, Pablo Picasso (Om...",Nearly everyone is miscast in this disjointed ...,78.0,4.0,74.0
...,...,...,...,...,...
2330,Awkward teen Adam spends his last high school ...,Much like its well-meaning but clueless protag...,8.0,74.0,-66.0
2710,Amy Schumer gives a refreshingly honest and hi...,Amy Schumer's standup matures in an hour fille...,9.0,79.0,-70.0
7452,NaN,Hannah Gadsby: Nanette brilliantly moves moder...,25.0,100.0,-75.0
9079,"A young bartender in the Bronx, a coal miner's...",A galvanizing glimpse behind the scenes of a p...,20.0,99.0,-79.0
